In [ ]:
import gc  
import os  
import time  
import warnings 
from itertools import combinations  
from warnings import simplefilter 
import joblib  
import playground.optivarfuncs as of
import lightgbm as lgb  
import numpy as np  
import pandas as pd  
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold, TimeSeriesSplit  
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
is_offline = False 
is_train = True  
is_infer = True 
max_lookback = np.nan 
split_day = 435  
import polars as pl
# set the max columns to none
pd.set_option('display.max_columns', None)

In [ ]:
import pandas as pd
import numpy as np
import itertools
def getdfstats(numb_days= 1, stocks=list(range(200)), numb_secs_day= 55):
    '''
    generate a dataframe to hold latest stock near and far prices
    num_days: how many days in dataframe
    stocks: stock ids
    n_nulls: first n rows per day have near and far price = NaN
    drop_days: remove these days from dataframe
    '''
    stockl = np.concatenate(numb_days*[list(itertools.repeat(n, numb_secs_day)) for n in stocks], axis=0).tolist()
    dayl = [np.trunc(d/(numb_secs_day*len(stocks))) for d in list(range(numb_days*numb_secs_day*len(stocks)))]
    vall = [0 for i in list(range(len(stocks)*numb_days*numb_secs_day))]
    secl = [(d % numb_secs_day)*10 for d in list(range(numb_days*numb_secs_day*len(stocks)))]

    df = pd.DataFrame(data={'stock_id': stockl, 'date_id': dayl, 'far_price': vall, 'near_price': vall, 'seconds_in_bucket': secl,'dummynp': vall,'dummyfp': vall})
    return df

# getdfstats()

In [ ]:
alldata=pd.read_csv('data/train.csv')

In [ ]:
import playground.optivarfuncs as of

def fillrow(row,dfstats):
    #if NaN in row.near_price or row.far_price then replace it with 
    #value in dfstats for the corresponding stock_id and seconds in bucket slot
    #if not NaN then save otherwise save row.near_price and row.far_price in dfstats
    # ex. df.apply(fillrow, axis=1, args=(dfstats,))

    rowstats=(dfstats.stock_id==row.stock_id) &(dfstats.seconds_in_bucket==row.seconds_in_bucket)
    
    if row.near_price==np.NaN:
        row.near_price=dfstats.loc[rowstats,'near_price']
        dfstats.loc[rowstats,'dummynp']=0
    else:
        dfstats.loc[rowstats,'near_price']=row.near_price
        dfstats.loc[rowstats,'dummynp']=1
    
    if row.far_price==np.NaN:
        row.far_price=dfstats.loc[rowstats,'far_price']
        dfstats.loc[rowstats,'dummyfp']=0
    else:
        dfstats.loc[rowstats,'far_price']=row.far_price
        dfstats.loc[rowstats,'dummyfp']=1
    return row
 
  
def interp(df,dfstats):
    #dataframe operated on
    #dfstats - dataframe with stats for each stock
    #max seconds in bucket is 540, once you hit this then recalculate all 54 values
    #for the next day
    if(df.iloc[0,'seconds_in_bucket']==540):
        #for each stock
        for stock in dfstats.stock_id.unique():
            #interpolate near price
            index=dfstats[((dfstats.stock_id==stock)&(dfstats.dummynp==1))].index
            low=dfstats.iloc[index.min()].near_price
            dfstats.iloc[index[0]]=low
            df.interpolate(limit=1, limit_direction='backward')
            dfstats[(dfstats.stock_id==stock)].dummynp=0

            #interpolate far price
            index=dfstats[((dfstats.stock_id==stock)&(dfstats.dummyfp==1))].index
            low=dfstats.iloc[index.min()].near_price
            dfstats.iloc[index[0]]=low
            df.interpolate(limit=1, limit_direction='backward')
            dfstats[(dfstats.stock_id==stock)].dummyfp=0


def estimate(df,dfstats):
    #df - dataframe that needs near_price and far price interpolated
    #dfstats- dataframe with stats for each stock

    df.apply(fillrow, axis=1, args=(dfstats,))
    interp(df,dfstats)
    return df,dfstats


dfstats=getdfstats()

#get the last day of data for all stocks(200stocks*55 seconds_in_bucket per stock)
cache=alldata[-11000:]

#use the cache to populate dfstats
_,dfstats=estimate(cache,dfstats)

#get some test data
df=getdfstats(numb_secs_day=1)

#see if we can interpolate
estimate(df,dfstats)  


In [ ]:
from collections import defaultdict 

from dataclasses import dataclass
@dataclass
class Stock:
    #whenever a stock has a non null value add it to this dict 
    #in the form vals[seconds_in_bucket]=price
    near_price_vals=defaultdict(lambda: 0)
    far_price_vals=defaultdict(lambda: 0)

class AllStocks: 
    def __init__(self):
        self.allstocks=defaultdict(lambda: Stock)
    def __getitem__(self,stock_id):
        return self.allstocks[stock_id]
    
seconds_in_bucket=10
stock_id=0
a=AllStocks()
seconds_in_bucket=10

v1=a[stock_id].near_price_vals[seconds_in_bucket]
a[stock_id].near_price_vals[seconds_in_bucket]=100
v1=a[stock_id].near_price_vals[seconds_in_bucket]

# Playground

In [ ]:
#get the last row for all stocks (should be 200 rows if no stocks delisted)
latest_date=df.date_id.max()
latest_seconds_in_bucket=df[df.date_id==latest_date].seconds_in_bucket.max()
dftmp=df[(df.date_id==latest_date) & (df.seconds_in_bucket==latest_seconds_in_bucket)]
dftmp
#save the values for near and far price for those stocks




In [ ]:
df = pd.read_csv("./data/train.csv")
df = df.dropna(subset=["target"])  #drop all rows with NaN in target
df.sort_values(by=['stock_id','date_id','seconds_in_bucket'],inplace=True)
df[109:141]

In [ ]:
df1=df.copy()
# dfgb=df.groupby(['stock_id','date_id']).near_price.mean()

#first do a forward interpolation
df1.far_price=df1.far_price.interpolate(method='linear')
df1.near_price=df1.near_price.interpolate(method='linear')
# df1[109:141]